# Interactive Plots 

## Active Interactions: Checkboxes and Sliders

In [14]:
import pandas as pd
import numpy as np
import datetime

In [15]:
from bokeh.io import *
from bokeh.plotting import *

from bokeh.models import *
from bokeh.models.widgets import *

from bokeh.layouts import *
from bokeh.palettes import *

from bokeh.application.handlers import *
from bokeh.application import *

output_notebook()

# Reading in Data

In [16]:
# Load in flights and inspect
data = pd.read_csv('project_data.csv', parse_dates=['DATE'])
print(data.head(), data.info())
available_people = ['merissa_steps', 'sheridan_steps', 'tova_steps']
data = data.melt(id_vars=['DATE', 'temp'], value_vars=available_people, var_name='person', value_name='steps')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 8 columns):
DATE              122 non-null datetime64[ns]
merissa_dist      122 non-null float64
merissa_steps     122 non-null float64
sheridan_dist     122 non-null float64
sheridan_steps    122 non-null float64
tova_dist         122 non-null float64
tova_steps        122 non-null float64
temp              122 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 7.8 KB
        DATE  merissa_dist  merissa_steps  sheridan_dist  sheridan_steps  \
0 2019-06-01      4.560388    10700.15590       3.674949     8663.697420   
1 2019-06-02      3.315483     7692.00000       2.793206     6468.595866   
2 2019-06-03      5.412403    11560.00000       0.846568     1620.272716   
3 2019-06-04      3.721314     7334.00000       0.747559     1737.605581   
4 2019-06-05      4.872565    10099.04567       1.117429     2359.269982   

   tova_dist  tova_steps  temp  
0   0.008152        

In [17]:
colors = []
for color in ['Tomato', 'Turquoise', 'Violet']:
    for entry in range(122):
        colors.append(color)
data['color_by_person'] = colors
# Added the column for week - Merissa
data['week'] = data.DATE.dt.week

In [18]:
data[data["week"] == 23]

,DATE,temp,person,steps,color_by_person,week
2,2019-06-03,70,merissa_steps,11560.000000,Tomato,23
3,2019-06-04,70,merissa_steps,7334.000000,Tomato,23
4,2019-06-05,81,merissa_steps,10099.045670,Tomato,23
5,2019-06-06,83,merissa_steps,9.594439,Tomato,23
6,2019-06-07,81,merissa_steps,9.594439,Tomato,23
7,2019-06-08,81,merissa_steps,9.594439,Tomato,23
8,2019-06-09,78,merissa_steps,9.594439,Tomato,23
124,2019-06-03,70,sheridan_steps,1620.272716,Turquoise,23
125,2019-06-04,70,sheridan_steps,1737.605581,Turquoise,23
126,2019-06-05,81,sheridan_steps,2359.269982,Turquoise,23


In [19]:
weekly_plots = []
for week in range(23,39):
    weekly = data[data["week"] == week]
    p = figure(plot_width=350, plot_height=200,
               x_axis_type='datetime',
               x_axis_label='Day', y_axis_label='Steps',
               title='Week {}'.format(week),
               toolbar_location='left', tools=['hover'], 
               tooltips=[("Name", "$name"),("Steps", "$y{int}")])
    for who, name, color in zip(["merissa_steps","sheridan_steps","tova_steps"],["Merissa","Sheridan","Tova"],['blue','orange','green']):    
        per = weekly[weekly['person'] == who]
        p.line(per.DATE, per.steps, color=color, legend=name, name=name)
    p.legend.click_policy='hide'
    weekly_plots.append(p)  
show(gridplot(weekly_plots, ncols=4))

In [20]:
data.head()

,DATE,temp,person,steps,color_by_person,week
0,2019-06-01,80,merissa_steps,10700.15590,Tomato,22
1,2019-06-02,81,merissa_steps,7692.00000,Tomato,22
2,2019-06-03,70,merissa_steps,11560.00000,Tomato,23
3,2019-06-04,70,merissa_steps,7334.00000,Tomato,23
4,2019-06-05,81,merissa_steps,10099.04567,Tomato,23


## Plot with Person Select Control

In [21]:
# These are the column names
available_people = ['merissa_steps', 'sheridan_steps', 'tova_steps']

def modify_doc(doc):
   
    def make_dataset(person_list):
        
        list_of_frames = []
        for person, frame in zip(person_list, 'abc'[:len(person_list)]):
            frame = data[data['person'] == person]
            list_of_frames.append(frame)
        by_person = pd.concat(list_of_frames)
    
        return ColumnDataSource(by_person)

    def tovas_plot(src):
        # Blank plot with correct labels
        p1 = figure(x_axis_type="datetime", title="Everyone's Steps", plot_height=200, plot_width=500,
                    tools=['box_select', 'reset', 'xwheel_zoom', 'xpan'])
        p1.scatter(source=src, x='DATE', y='steps', line_color='color_by_person', fill_color='color_by_person')
        #for col in src.column_names[1:]:
        #    p.line(source = src, x = 'DATE', y = col, color='gray')

        return p1

    def sheridans_plot(src):
        # Blank plot with correct labels
        p2 = figure(x_axis_type="datetime", title="Steps by temperature", plot_height=200, plot_width=500,
                    tools=['box_select', 'reset'])
        p2.scatter(source=src, x='temp', y='steps', line_color='color_by_person', fill_color='color_by_person')
        #for col in src.column_names[1:]:
        #    p.line(source = src, x = 'DATE', y = col, color='gray')

        return p2    

    def merissas_plot(src): 
        weekly_plots = []
        for week in range(23,39):
# SUBSET DATA HERE BY WEEK
            #weekly = src[src["week"] == week]
            p3 = figure(plot_width=350, plot_height=200,
                        x_axis_type='datetime',
                        x_axis_label='Day', y_axis_label='Steps',
                        title='Week {}'.format(week),
                        toolbar_location='left', tools=['hover'], 
                        tooltips=[("Name", "$name"),("Steps", "$y{int}")])
            p3.scatter(source=src, x="DATE", y="steps", line_color='color_by_person', fill_color='color_by_person')
            weekly_plots.append(p3)
        gp = gridplot(weekly_plots, ncols=4)
        return gp
    
    
    # Update function takes three default parameters
    def update(attr, old, new):
        # Get the list of carriers for the graph
        people_to_plot = [person_selection.labels[i] for i in 
                            person_selection.active]
    
        # Make a new dataset based on the selected carriers and the 
        # make_dataset function defined earlier
        new_src = make_dataset(people_to_plot)

        # Update the source used the quad glpyhs
        src.data.update(new_src.data)
    
    
    
    person_selection = CheckboxGroup(labels=available_people, active = [0, 1, 2])
    person_selection.on_change('active', update)
    
    controls = WidgetBox(person_selection)
    
    initial_people = [person_selection.labels[i] for i in person_selection.active]
    
    src = make_dataset(initial_people)
    
    p1 = tovas_plot(src)
    p2 = sheridans_plot(src)
    gp = merissas_plot(src)

    layout = row(column(controls, p1, p2), gp)
    doc.add_root(layout)
    
# Set up an application
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [22]:
show(app)